In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: ignored

In [ ]:
# %cd /content/gdrive/MyDrive/YOLOv7_pose
# !git clone https://github.com/RizwanMunawar/yolov7-pose-estimation.git

/content/gdrive/MyDrive/YOLOv7_pose
Cloning into 'yolov7-pose-estimation'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 166 (delta 56), reused 68 (delta 24), pack-reused 62
Receiving objects: 100% (166/166), 3.75 MiB | 15.48 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [ ]:
!nvidia-smi

Thu Mar 16 02:34:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    31W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd /content/gdrive/MyDrive/YOLOv7_pose/yolov7-pose-estimation
!pip install -r requirements.txt

/content/gdrive/MyDrive/YOLOv7_pose/yolov7-pose-estimation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.9 MB/s eta 0:00:00


In [ ]:
!python pose-estimate.py --source "/content/gdrive/MyDrive/Datasets/Non-Cheating/Non-cheating_248.mp4" --device 0

2023-03-16 03:05:46.802444: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 03:05:47.762845: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.9/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2023-03-16 03:05:47.763000: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.9/dist-packages/cv2/../../lib64:/u

In [ ]:
import numpy as np
import pandas as pd
from keras.layers import LSTM, Dense,Dropout
from keras.models import Sequential
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv("test.csv")
df

,kp0_X,kp0_Y,kp0_Confi,kp1_X,kp1_Y,kp1_Confi,kp2_X,kp2_Y,kp2_Confi,kp3_X,...,kp14_X,kp14_Y,kp14_Confi,kp15_X,kp15_Y,kp15_Confi,kp16_X,kp16_Y,kp16_Confi,label
0,0.627783,0.856736,0.923219,0.702739,0.880992,0.897338,0.575966,0.895504,0.880142,0.823285,...,0.000000,0.304594,0.866050,0.668223,0.052104,0.500082,0.053231,0.000000,0.514520,Non-Cheating
1,0.630523,0.858777,0.909266,0.703706,0.883392,0.874186,0.579122,0.896659,0.861140,0.820322,...,0.000000,0.304502,0.865908,0.000000,0.000000,0.000000,0.054519,0.000000,0.511976,Non-Cheating
2,0.628696,0.858738,0.928549,0.702393,0.883721,0.903198,0.576931,0.897197,0.889852,0.820132,...,0.000000,0.307547,0.863121,0.000000,0.000000,0.000000,0.051829,0.000000,0.503234,Non-Cheating
3,0.625742,0.857248,0.931711,0.699985,0.882158,0.912199,0.572713,0.895703,0.892731,0.820457,...,0.000000,0.303982,0.866609,0.000000,0.000000,0.000000,0.053558,0.000000,0.510904,Non-Cheating
4,0.626030,0.857502,0.921093,0.699691,0.882271,0.895631,0.573013,0.895215,0.878399,0.817954,...,0.000000,0.300596,0.867966,0.000000,0.000000,0.000000,0.051245,0.000000,0.512254,Non-Cheating
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5936,0.409495,0.858140,0.891660,0.479993,0.895990,0.927850,0.342490,0.902327,0.786151,0.647598,...,0.118642,0.243210,0.887838,0.790974,0.000000,0.686193,0.404293,0.039333,0.687370,Non-Cheating
5937,0.408809,0.858281,0.889903,0.477956,0.896134,0.925658,0.342450,0.902025,0.779671,0.645722,...,0.106124,0.242601,0.887305,0.793491,0.000000,0.687415,0.405893,0.038556,0.688159,Non-Cheating
5938,0.411675,0.859023,0.878882,0.481305,0.896417,0.915483,0.345015,0.902216,0.757801,0.649042,...,0.106038,0.257244,0.898706,0.791096,0.000000,0.719262,0.393406,0.051535,0.719531,Non-Cheating
5939,0.407110,0.859503,0.875994,0.477513,0.896373,0.915189,0.341427,0.903685,0.754751,0.648741,...,0.134676,0.246024,0.899425,0.793454,0.000000,0.723523,0.421999,0.043532,0.724942,Non-Cheating


In [ ]:
df_cheating = df[df['label'] == 'Cheating']
df_noncheating = df[df['label'] == 'Non-Cheating']

In [ ]:
df_cheating

,kp0_X,kp0_Y,kp0_Confi,kp1_X,kp1_Y,kp1_Confi,kp2_X,kp2_Y,kp2_Confi,kp3_X,...,kp14_X,kp14_Y,kp14_Confi,kp15_X,kp15_Y,kp15_Confi,kp16_X,kp16_Y,kp16_Confi,label
1506,0.617659,0.839054,0.863150,0.684358,0.875559,0.844554,0.563378,0.886501,0.764939,0.798646,...,0.0,0.013405,0.792924,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
1507,0.618699,0.840609,0.875331,0.684703,0.878015,0.855298,0.564693,0.888477,0.785752,0.796680,...,0.0,0.008516,0.796468,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
1508,0.622940,0.843531,0.846056,0.687709,0.880058,0.815234,0.569388,0.889436,0.746726,0.797397,...,0.0,0.023587,0.795436,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
1509,0.620980,0.836386,0.881042,0.687272,0.873766,0.862819,0.566025,0.883769,0.797772,0.798552,...,0.0,0.011816,0.787432,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
1510,0.619537,0.838653,0.887448,0.685610,0.876426,0.871103,0.564638,0.886251,0.806280,0.797854,...,0.0,0.019424,0.794113,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4434,0.552702,0.786243,0.959089,0.632845,0.826189,0.965031,0.510515,0.881403,0.879289,0.802775,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
4435,0.553368,0.734863,0.964105,0.633271,0.773503,0.969266,0.510315,0.823967,0.895836,0.802654,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
4436,0.551926,0.732366,0.963998,0.632181,0.769907,0.968926,0.509125,0.822655,0.895965,0.803945,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Cheating
4437,0.551607,0.771825,0.962581,0.631423,0.810368,0.967919,0.508856,0.866792,0.892896,0.803266,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,Cheating


In [ ]:
df_noncheating

,kp0_X,kp0_Y,kp0_Confi,kp1_X,kp1_Y,kp1_Confi,kp2_X,kp2_Y,kp2_Confi,kp3_X,...,kp14_X,kp14_Y,kp14_Confi,kp15_X,kp15_Y,kp15_Confi,kp16_X,kp16_Y,kp16_Confi,label
0,0.627783,0.856736,0.923219,0.702739,0.880992,0.897338,0.575966,0.895504,0.880142,0.823285,...,0.000000,0.304594,0.866050,0.668223,0.052104,0.500082,0.053231,0.000000,0.514520,Non-Cheating
1,0.630523,0.858777,0.909266,0.703706,0.883392,0.874186,0.579122,0.896659,0.861140,0.820322,...,0.000000,0.304502,0.865908,0.000000,0.000000,0.000000,0.054519,0.000000,0.511976,Non-Cheating
2,0.628696,0.858738,0.928549,0.702393,0.883721,0.903198,0.576931,0.897197,0.889852,0.820132,...,0.000000,0.307547,0.863121,0.000000,0.000000,0.000000,0.051829,0.000000,0.503234,Non-Cheating
3,0.625742,0.857248,0.931711,0.699985,0.882158,0.912199,0.572713,0.895703,0.892731,0.820457,...,0.000000,0.303982,0.866609,0.000000,0.000000,0.000000,0.053558,0.000000,0.510904,Non-Cheating
4,0.626030,0.857502,0.921093,0.699691,0.882271,0.895631,0.573013,0.895215,0.878399,0.817954,...,0.000000,0.300596,0.867966,0.000000,0.000000,0.000000,0.051245,0.000000,0.512254,Non-Cheating
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5936,0.409495,0.858140,0.891660,0.479993,0.895990,0.927850,0.342490,0.902327,0.786151,0.647598,...,0.118642,0.243210,0.887838,0.790974,0.000000,0.686193,0.404293,0.039333,0.687370,Non-Cheating
5937,0.408809,0.858281,0.889903,0.477956,0.896134,0.925658,0.342450,0.902025,0.779671,0.645722,...,0.106124,0.242601,0.887305,0.793491,0.000000,0.687415,0.405893,0.038556,0.688159,Non-Cheating
5938,0.411675,0.859023,0.878882,0.481305,0.896417,0.915483,0.345015,0.902216,0.757801,0.649042,...,0.106038,0.257244,0.898706,0.791096,0.000000,0.719262,0.393406,0.051535,0.719531,Non-Cheating
5939,0.407110,0.859503,0.875994,0.477513,0.896373,0.915189,0.341427,0.903685,0.754751,0.648741,...,0.134676,0.246024,0.899425,0.793454,0.000000,0.723523,0.421999,0.043532,0.724942,Non-Cheating


In [ ]:
columnName = []
for i in range(17):
  columnName.append("kp" + str(i) + "_X")
  columnName.append("kp" + str(i) + "_Y")
  columnName.append("kp" + str(i) + "_Confi")

In [ ]:
no_of_timesteps = 30
n_sample = len(df_cheating)
df_cheating = np.resize(df_cheating[columnName],(int(n_sample/no_of_timesteps), no_of_timesteps, 51))

n_sample = len(df_noncheating)
df_noncheating = np.resize(df_noncheating[columnName],(int(n_sample/no_of_timesteps), no_of_timesteps, 51))

df_cheating

array([[[0.61765935, 0.83905355, 0.8631503 , ..., 0.        ,
         0.        , 0.        ],
        [0.61869927, 0.84060884, 0.87533104, ..., 0.        ,
         0.        , 0.        ],
        [0.62293992, 0.84353055, 0.8460564 , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.63012499, 0.83525905, 0.87172413, ..., 0.        ,
         0.        , 0.        ],
        [0.62412254, 0.83248481, 0.88947695, ..., 0.        ,
         0.        , 0.        ],
        [0.62268056, 0.84039419, 0.87712914, ..., 0.        ,
         0.        , 0.        ]],

       [[0.62738811, 0.83625691, 0.88208091, ..., 0.        ,
         0.        , 0.        ],
        [0.62829331, 0.83793861, 0.90317792, ..., 0.        ,
         0.        , 0.        ],
        [0.62817675, 0.83510307, 0.89245373, ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.37233222, 0.94879779, 0.93175489, ..., 0.        ,
         0.        , 0.        ],
        [0.3

In [ ]:
y = []
print(df_cheating.shape[0])
print(df_noncheating.shape[0])
for i in range(df_cheating.shape[0]):
  y.append(0)
for i in range(df_noncheating.shape[0]):
  y.append(1)
print(y)

97
100
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
Y = df['label']
lc = LabelEncoder()
lc = lc.fit(Y)
Y = lc.transform(Y)
print(Y)

(4439,)
[1 1 1 ... 0 0 0]


In [ ]:
X = np.append(df_cheating,df_noncheating , axis=0)
print(X)

[[[0.61765935 0.83905355 0.8631503  ... 0.         0.         0.        ]
  [0.61869927 0.84060884 0.87533104 ... 0.         0.         0.        ]
  [0.62293992 0.84353055 0.8460564  ... 0.         0.         0.        ]
  ...
  [0.63012499 0.83525905 0.87172413 ... 0.         0.         0.        ]
  [0.62412254 0.83248481 0.88947695 ... 0.         0.         0.        ]
  [0.62268056 0.84039419 0.87712914 ... 0.         0.         0.        ]]

 [[0.62738811 0.83625691 0.88208091 ... 0.         0.         0.        ]
  [0.62829331 0.83793861 0.90317792 ... 0.         0.         0.        ]
  [0.62817675 0.83510307 0.89245373 ... 0.         0.         0.        ]
  ...
  [0.37233222 0.94879779 0.93175489 ... 0.         0.         0.        ]
  [0.33963236 0.94282793 0.94807988 ... 0.         0.         0.        ]
  [0.38380125 0.93153642 0.91417474 ... 0.         0.         0.        ]]

 [[0.38339651 0.92553078 0.90347087 ... 0.         0.         0.        ]
  [0.41098766 0.907238

In [ ]:
y = np.array(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model  = Sequential()
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
model.add(Dense(units = 1, activation="sigmoid"))
model.compile(optimizer="adam", metrics = ['accuracy'], loss = "binary_crossentropy")

model.fit(X_train, y_train, epochs=20, batch_size=32,validation_data=(X_test, y_test))
model.save("model.h5")

Epoch 1/20
5/5 [==============================] - 8s 331ms/step - loss: 0.6727 - accuracy: 0.5605 - val_loss: 0.6396 - val_accuracy: 0.6750
Epoch 2/20
5/5 [==============================] - 0s 21ms/step - loss: 0.6080 - accuracy: 0.6943 - val_loss: 0.5352 - val_accuracy: 0.7500
Epoch 3/20
5/5 [==============================] - 0s 21ms/step - loss: 0.4795 - accuracy: 0.7070 - val_loss: 0.4638 - val_accuracy: 0.8000
Epoch 4/20
5/5 [==============================] - 0s 19ms/step - loss: 0.4142 - accuracy: 0.7898 - val_loss: 0.4301 - val_accuracy: 0.7500
Epoch 5/20
5/5 [==============================] - 0s 18ms/step - loss: 0.3930 - accuracy: 0.8217 - val_loss: 0.3750 - val_accuracy: 0.7500
Epoch 6/20
5/5 [==============================] - 0s 19ms/step - loss: 0.3315 - accuracy: 0.8535 - val_loss: 0.4319 - val_accuracy: 0.8250
Epoch 7/20
5/5 [==============================] - 0s 19ms/step - loss: 0.3471 - accuracy: 0.8153 - val_loss: 0.3043 - val_accuracy: 0.7750
Epoch 8/20
5/5 [==========

In [ ]:
AllKeypoint = np.array([])
# AllKeypoint = np.array(
#     [
#         [ [] ],
#         [ [] ],
#         [ [] ],
#     ]
#     )
AllKeypoint = np.resize(AllKeypoint, (3, 1, 0))
AllKeypoint = np.append(AllKeypoint[1,:,:],[3])
print(AllKeypoint[1,:,:])

IndexError: ignored

In [ ]:
!python detect.py --source "/content/gdrive/MyDrive/Datasets/Cheating/Cheating41.mp4" --device 0

2023-01-30 07:17:01.211868: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Optimizer stripped from yolov7-w6-pose.pt, 161.1MB
Fusing layers... 
/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Frame 0 Processing
Frame 1 Processing
Frame 2 Processing
Frame 3 Processing
Frame 4 Processing
Frame 5 Processing
Frame 6 Processing
Frame 7 Processing
Frame 8 Processing
Frame 9 Processing
Frame 10 Processing
Frame 11 Processing
Frame 12 Processing
Frame 13 Processing
Frame 14 Processing
Frame 15 Processing
Frame 16 Processing
Frame 17 Processing
Frame 18 Processing
Frame 19 Processing
Frame 20 Proce

In [ ]:
# Array=[]
# Array = np.array(Array)
model1 = tf.keras.models.load_model("model.h5")
test = np.expand_dims(X_test[1,:,:],axis=0)
np.array(test)
print(test.shape)
print(X_test[1,:,:])

results = model1.predict(np.resize(X_test[1,:,:],(1, 30, 51)))
print(results)

(1, 30, 51)
[[0.51415132 0.75126957 0.98361671 ... 0.         0.         0.        ]
 [0.51190031 0.74833946 0.98550874 ... 0.         0.         0.        ]
 [0.50810921 0.74654626 0.98252827 ... 0.         0.         0.        ]
 ...
 [0.51762441 0.74717886 0.97122365 ... 0.         0.         0.        ]
 [0.51910433 0.75041206 0.97411591 ... 0.         0.         0.        ]
 [0.51760186 0.75021713 0.97812265 ... 0.         0.         0.        ]]
1/1 [==============================] - 1s 1s/step
[[0.30750576]]


In [ ]:
label = ["Warmup...."]
for i in range(4):
  label.append("Warmup....")
label

['Warmup....', 'Warmup....', 'Warmup....', 'Warmup....', 'Warmup....']